<a href="https://colab.research.google.com/github/sahug/ds-tensorflow-colab/blob/master/Tensorflow%20-%20Google%20Stock%20Price%20Prediction%20Using%20RNN-LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Tensorflow - Google Stock Price Prediction Using RNN-LSTM**

**Steps**
 - Data Preprocessing
 - Building the RNN
 - Making predictions and Visualization

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

In [ ]:
data = pd.read_csv("data/GOOG.csv", date_parser=True)
data.head()

In [ ]:
data_training = data[data["Date"] < "2019-01-01"].copy()
data_training

In [ ]:
data_test = data[data["Date"] >= "2019-01-01"].copy()
data_test

In [ ]:
training_data = data_training.drop(["Date", "Adj Close"], axis=1)
training_data.head()

In [ ]:
scaler = MinMaxScaler()
training_data = scaler.fit_transform(training_data)
training_data

In [ ]:
x_train = []
y_train = []

In [ ]:
training_data.shape, training_data.shape[0]

In [ ]:
# Splitting data in chunks of 60
for i in range(60, training_data.shape[0]):
    x_train.append(training_data[i - 60 : i])
    y_train.append(
        training_data[i, 0]
    )  # Open is what we are pridicting or is our outcome

In [ ]:
x_train, y_train = np.array(x_train), np.array(y_train)

In [ ]:
x_train.shape, y_train.shape

**Building LSTM**

In [ ]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

In [ ]:
x_train.shape[1], 5

In [ ]:
regressior = Sequential()

regressior.add(
    LSTM(
        units=50,
        activation="relu",
        return_sequences=True,
        input_shape=(x_train.shape[1], 5),
    )
)
regressior.add(Dropout(0.2))

regressior.add(LSTM(units=60, activation="relu", return_sequences=True))
regressior.add(Dropout(0.3))

regressior.add(LSTM(units=80, activation="relu", return_sequences=True))
regressior.add(Dropout(0.4))

regressior.add(LSTM(units=120, activation="relu"))
regressior.add(Dropout(0.3))

regressior.add(Dense(units=1))

In [ ]:
regressior.summary()

In [ ]:
regressior.compile(optimizer="adam", loss="mean_squared_error")

In [ ]:
history = regressior.fit(x_train, y_train, epochs=10, batch_size=32)

**Prepare Test Dataset**

In [ ]:
data_test.head()

To predict the Open price of the stock for the current data we need to look at the last 60 days data.

In [ ]:
past_60_days = data_training.tail(60)

In [ ]:
df = past_60_days.append(data_test, ignore_index=True)
df = df.drop(["Date", "Adj Close"], axis=1)
df.head()

In [ ]:
inputs = scaler.transform(df)
inputs

In [ ]:
x_test = []
y_test = []

for i in range(60, inputs.shape[0]):
    x_test.append(inputs[i - 60 : i])
    y_test.append(inputs[i, 0])

In [ ]:
x_test, y_test = np.array(x_test), np.array(y_test)
x_test.shape, y_test.shape

In [ ]:
y_pred = regressior.predict(x_test)

In [ ]:
y_pred

**Reverse Scaling to Original Values**

In [ ]:
scaler.scale_

In [ ]:
scale = 1 / 8.18605127e-04
scale

In [ ]:
y_pred = y_pred * scale
y_test = y_test * scale

In [ ]:
y_pred

In [ ]:
y_test

**Visualization**

In [ ]:
plt.figure(figsize=(14, 5))
plt.plot(y_test, color="red", label="Real Google Stock Price")
plt.plot(y_pred, color="blue", label="Predicted Google Stock Price")
plt.title("Google Stock Price Prediction")
plt.xlabel("Time")
plt.ylabel("Google Stock Price")
plt.legend()
plt.show()